In [ ]:
import hvac
import pyspark.sql.functions as F  # noqa
import pyspark.sql.types as T  # noqa
from pyspark.sql import SparkSession

client = hvac.Client(url="http://vault.svc.internal:8200")
client.auth.userpass.login(username="admin", password="vaultadmin")

# Note. Spark Session 구성
spark = (
    SparkSession.builder.appName("")
    .config("spark.sql.caseSensitive", True)
    .config("spark.sql.session.timeZone", "UTC")
    .config("spark.sql.parquet.datetimeRebaseModeInWrite", "CORRECTED")
    .config("spark.sql.parquet.datetimeRebaseModeInRead", "CORRECTED")
    .config("spark.sql.parquet.int96RebaseModeInRead", "CORRECTED")
    .config("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")
    .getOrCreate()
)

In [ ]:
# Note. MySQL 인증 정보
secret = client.read("secret/data/user/database/local-mysql")["data"]["data"]
DATABASE_OPTS = {
    "url": f"jdbc:mysql://{secret['host']}:{secret['port']}",
    "user": secret["user"],
    "password": secret["password"],
    "driver": "com.mysql.cj.jdbc.Driver",
}

In [ ]:
# Note. MSSQL  인증 정보
secret = client.read("secret/data/user/database/local-sqlserver")["data"]["data"]
database = "store"
DATABASE_OPTS = {
    "url": f"jdbc:sqlserver://{secret['host']}:{secret['port']};databaseName={database};encrypt=false;",
    "user": secret["user"],
    "password": secret["password"],
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver",
}

In [ ]:
df = (spark.read.format("jdbc").options(**DATABASE_OPTS).option("query", "select * from store.tb_lower").load()).show(
    truncate=False
)